
# 차원 축소 알고리즘: PCA, t-SNE, UMAP
차원 축소는 고차원 데이터를 저차원으로 변환하여 데이터의 구조를 더 잘 이해하고 시각화하는 데 도움을 주는 기법입니다. 이 글에서는 대표적인 차원 축소 알고리즘인 PCA, t-SNE, UMAP에 대해 정리하고, 각각의 알고리즘의 방법론, 장단점, 그리고 논문에 대해 소개합니다.


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap

# 데이터 로드
data = pd.read_csv('/mnt/data/creditcard.csv')

# 데이터 기본 정보 확인
data.info()
data.describe()


In [ ]:

# 데이터의 첫 몇 행 확인
data.head()



## Class 분포 확인
Class 컬럼의 분포를 시각화하여 데이터의 불균형 여부를 확인합니다.


In [ ]:

plt.figure(figsize=(8, 4))
sns.countplot(x='Class', data=data)
plt.title('Class Distribution')
plt.show()



## 특성 간 상관 관계 히트맵
데이터의 특성 간 상관 관계를 히트맵으로 시각화하여 중요한 특성을 확인합니다.


In [ ]:

plt.figure(figsize=(16, 12))
sns.heatmap(data.corr(), cmap='coolwarm', annot=False)
plt.title('Feature Correlation Matrix')
plt.show()



## 데이터 전처리 및 스케일링
Class 컬럼을 제외한 데이터의 스케일링을 수행합니다.


In [ ]:

X = data.drop('Class', axis=1)
y = data['Class']

# 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)



# 1. PCA (Principal Component Analysis)
## 방법론
PCA는 공분산 행렬을 기반으로 하는 matrix factorization 기법입니다. 고차원 데이터의 분산이 최대인 축을 찾아 그 축으로 데이터를 투영합니다. 첫 번째 주성분은 분산이 최대인 방향을 나타내고, 두 번째 주성분은 첫 번째 주성분과 직교하면서 분산이 최대인 방향을 나타냅니다. 이렇게 주성분들을 구하여 데이터를 저차원으로 변환합니다.

## 장점
- 간단하고 직관적입니다.
- 데이터의 분산을 최대한 보존합니다.
- 노이즈를 줄이는 데 효과적입니다.

## 단점
- 선형 변환에 기반하므로 비선형 구조를 잘 반영하지 못합니다.
- 데이터가 뭉게질 수 있습니다.

## 참고 논문
- Pearson, K. (1901). "On Lines and Planes of Closest Fit to Systems of Points in Space". Philosophical Magazine.

### 파이썬 코드 예시
PCA를 사용하여 데이터를 2차원으로 축소하고 시각화합니다.


In [ ]:

pca = PCA(n_components=2)
pca_result = pca.fit_transform(X_scaled)

plt.figure(figsize=(10, 6))
plt.scatter(pca_result[:, 0], pca_result[:, 1], c=y, cmap='viridis', alpha=0.5)
plt.title('PCA Result')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.colorbar(label='Class')
plt.show()



PCA 결과를 해석하면, 데이터의 주성분이 최대 분산을 가진 방향으로 투영되어 있는 것을 볼 수 있습니다. 이 결과는 데이터의 전체적인 분포를 이해하는 데 도움이 되며, 주요 변동성을 나타내는 두 가지 축을 제공합니다.



# 2. t-SNE (t-distributed Stochastic Neighbor Embedding)
## 방법론
t-SNE는 데이터의 국소적 이웃 구조를 보존하는 neighbor graph 기반의 기법입니다. 고차원 데이터의 유사성을 저차원에서도 유지하도록 변환합니다. t 분포를 사용하여 하나의 기준점과 다른 모든 데이터 간의 거리를 계산하고, 유사한 데이터끼리 묶어줍니다.

## 장점
- 데이터의 특징을 잘 가시화할 수 있습니다.
- PCA와 같은 선형 변환 기법에서 발생하는 데이터 뭉개짐 문제를 해결합니다.
- 하이퍼파라미터의 영향이 적고 이상치에 둔감합니다.

## 단점
- 연산 시간이 오래 걸립니다.
- 높은 차원에서 바로 저차원으로 축소하기 어렵습니다.
- 매번 실행할 때마다 다른 결과가 나올 수 있습니다.
- 정보 손실로 인해 데이터 왜곡 가능성이 있습니다.

## 참고 논문
- Maaten, L. v. d., & Hinton, G. (2008). "Visualizing Data using t-SNE". Journal of Machine Learning Research.

### 파이썬 코드 예시
t-SNE를 사용하여 데이터를 2차원으로 축소하고 시각화합니다.


In [ ]:

tsne = TSNE(n_components=2, random_state=42)
tsne_result = tsne.fit_transform(X_scaled)

plt.figure(figsize=(10, 6))
plt.scatter(tsne_result[:, 0], tsne_result[:, 1], c=y, cmap='viridis', alpha=0.5)
plt.title('t-SNE Result')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.colorbar(label='Class')
plt.show()



# 3. UMAP (Uniform Manifold Approximation and Projection)
## 방법론
UMAP은 neighbor graph 기반의 기법으로, 고차원 공간의 데이터를 그래프로 만든 후 저차원으로 투영합니다. 이 과정은 리만 기하학과 위상수학에 기반하여 수학적으로 유효함이 증명되어 있습니다.

## 장점
- 빠른 연산 속도
- 임베딩 차원의 크기에 대한 제한이 없습니다.
- global structure를 잘 보존합니다.
- 탄탄한 이론적 배경

## 단점
- 하이퍼파라미터의 영향을 크게 받습니다.
- 저차원 임베딩 시 데이터 간 거리가 실제 거리와 일치하지 않을 수 있습니다.
- 정보 손실로 인한 데이터 왜곡 가능성이 있습니다.

## 참고 논문
- McInnes, L., Healy, J., & Melville, J. (2018). "UMAP: Uniform Manifold Approximation and Projection for Dimension Reduction". arXiv preprint arXiv:1802.03426.

### 파이썬 코드 예시
UMAP을 사용하여 데이터를 2차원으로 축소하고 시각화합니다.


In [ ]:

umap_reducer = umap.UMAP(n_components=2, random_state=42)
umap_result = umap_reducer.fit_transform(X_scaled)

plt.figure(figsize=(10, 6))
plt.scatter(umap_result[:, 0], umap_result[:, 1], c=y, cmap='viridis', alpha=0.5)
plt.title('UMAP Result')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.colorbar(label='Class')
plt.show()



## 결론
PCA, t-SNE, UMAP은 각각 고유한 장단점을 가지며, 데이터의 특성과 분석 목적에 따라 적절한 알고리즘을 선택하는 것이 중요합니다. 고차원 데이터를 다루는 다양한 프로젝트에서 이들 기법을 활용하여 효과적으로 차원을 축소하고 데이터를 이해할 수 있습니다. 각각의 알고리즘에 대한 좀 더 심도있는 내용은 추후에 개별 포스트로 한번 작성해보도록 하겠습니다.

읽어주셔서 감사합니다 🤗
